####**Summary so far**

I have created 3 zones(**A,B,C**) and 3 subzones each within them(**'A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'**). The criteria to determine if the zone is **'safe' , 'at risk' or 'flooded'** are
**water_level,people_trapped, medics, boats and food_packs** . I then increment the situation by 2 steps and which changes the parameters, in turn updating the status of the areas. I generated the summary report. Then, I implemented rule based logic to determine what parameters should be deployed in **flooded** areas to control the flooding. I plan to compare this suggestion with the suggestion given by the LLM. Currently, I am facing difficulty in getting response from the LLM and would need more time for it and later fine tune the rule based logic  and try LLM with different models

In [ ]:
import random
import pandas as pd

# Create 3x3 Grid Zones
zones = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

# Initialize zone status
zone_state = {
    zone: {
        'water_level': random.randint(0, 2),     # 0 = dry, 1 = light, 2 = flooded
        'people_trapped': random.randint(0, 50),
        'medics': 0,
        'boats': 0,
        'food_packs': 0,
        'status': 'safe'  # could be 'safe', 'at-risk', 'flooded'
    } for zone in zones
}

# Display initial state
df = pd.DataFrame.from_dict(zone_state, orient='index')
print("Initial Flood Map:")
print(df)

# Function to simulate flood rising
def simulate_flood(state, time_step):
    print(f"\n--- Time Step {time_step}: Flood Progression ---")
    for zone, data in state.items():
        if random.random() < 0.5:  # 50% chance flood worsens
            data['water_level'] = min(data['water_level'] + 1, 3)
            data['people_trapped'] += random.randint(0, 20)
        # Update status
        if data['water_level'] >= 3:
            data['status'] = 'flooded'
        elif data['water_level'] == 2:
            data['status'] = 'at-risk'
        else:
            data['status'] = 'safe'
    return state

# Simulate 3 time steps
for t in range(1, 4):
    zone_state = simulate_flood(zone_state, t)
    df = pd.DataFrame.from_dict(zone_state, orient='index')
    display(df)


Initial Flood Map:
    water_level  people_trapped  medics  boats  food_packs status
A1            2               1       0      0           0   safe
A2            0               0       0      0           0   safe
A3            0              39       0      0           0   safe
B1            1              13       0      0           0   safe
B2            1              24       0      0           0   safe
B3            1              12       0      0           0   safe
C1            0              40       0      0           0   safe
C2            0              23       0      0           0   safe
C3            0              49       0      0           0   safe

--- Time Step 1: Flood Progression ---


,water_level,people_trapped,medics,boats,food_packs,status
A1,2,1,0,0,0,at-risk
A2,1,7,0,0,0,safe
A3,0,39,0,0,0,safe
B1,1,13,0,0,0,safe
B2,1,24,0,0,0,safe
B3,2,25,0,0,0,at-risk
C1,1,50,0,0,0,safe
C2,1,40,0,0,0,safe
C3,0,49,0,0,0,safe



--- Time Step 2: Flood Progression ---


,water_level,people_trapped,medics,boats,food_packs,status
A1,2,1,0,0,0,at-risk
A2,1,7,0,0,0,safe
A3,1,40,0,0,0,safe
B1,2,22,0,0,0,at-risk
B2,1,24,0,0,0,safe
B3,2,25,0,0,0,at-risk
C1,1,50,0,0,0,safe
C2,1,40,0,0,0,safe
C3,1,56,0,0,0,safe



--- Time Step 3: Flood Progression ---


,water_level,people_trapped,medics,boats,food_packs,status
A1,3,13,0,0,0,flooded
A2,1,7,0,0,0,safe
A3,2,51,0,0,0,at-risk
B1,2,22,0,0,0,at-risk
B2,1,24,0,0,0,safe
B3,3,45,0,0,0,flooded
C1,2,51,0,0,0,at-risk
C2,1,40,0,0,0,safe
C3,1,56,0,0,0,safe


In [ ]:
def generate_flood_summary(zone_state):
    report = "Current Flood Situation Report:\n"
    for zone, data in zone_state.items():
        report += (
            f"- Zone {zone}: Status - {data['status'].upper()}, "
            f"Water Level - {data['water_level']}, "
            f"People Trapped - {data['people_trapped']}, "
            f"Boats - {data['boats']}, "
            f"Medics - {data['medics']}, "
            f"Food Packs - {data['food_packs']}\n"
        )
    return report


In [ ]:
# Generate the flood report summary
flood_report = generate_flood_summary(zone_state)

# Display the report
print(flood_report)

Current Flood Situation Report:
- Zone A1: Status - FLOODED, Water Level - 3, People Trapped - 13, Boats - 0, Medics - 0, Food Packs - 0
- Zone A2: Status - SAFE, Water Level - 1, People Trapped - 7, Boats - 0, Medics - 0, Food Packs - 0
- Zone A3: Status - AT-RISK, Water Level - 2, People Trapped - 51, Boats - 0, Medics - 0, Food Packs - 0
- Zone B1: Status - AT-RISK, Water Level - 2, People Trapped - 22, Boats - 0, Medics - 0, Food Packs - 0
- Zone B2: Status - SAFE, Water Level - 1, People Trapped - 24, Boats - 0, Medics - 0, Food Packs - 0
- Zone B3: Status - FLOODED, Water Level - 3, People Trapped - 45, Boats - 0, Medics - 0, Food Packs - 0
- Zone C1: Status - AT-RISK, Water Level - 2, People Trapped - 51, Boats - 0, Medics - 0, Food Packs - 0
- Zone C2: Status - SAFE, Water Level - 1, People Trapped - 40, Boats - 0, Medics - 0, Food Packs - 0
- Zone C3: Status - SAFE, Water Level - 1, People Trapped - 56, Boats - 0, Medics - 0, Food Packs - 0



**Implementing rule based AI**

In [ ]:
from collections import defaultdict

def rule_based_response_weighted(zone_state, boats=4, medics=4, food_packs=4):
    response = defaultdict(lambda: {"boats": 0, "medics": 0, "food_packs": 0})

    # Step 1: Calculate weights based on people × water level
    zone_weights = {}
    total_weight = 0
    for zone, data in zone_state.items():
        weight = data['people_trapped'] * (data['water_level'] + 1)  # +1 to give dry zones minimal weight
        zone_weights[zone] = weight
        total_weight += weight

    if total_weight == 0:
        return dict(response)  # no need if no one is affected

    # Step 2: Allocate resources proportionally (floating)
    boat_alloc = {z: (zone_weights[z] / total_weight) * boats for z in zone_weights}
    medic_alloc = {z: (zone_weights[z] / total_weight) * medics for z in zone_weights}
    food_alloc = {z: (zone_weights[z] / total_weight) * food_packs for z in zone_weights}

    # Step 3: Round down all and track remaining
    remaining_boats = boats
    remaining_medics = medics
    remaining_food = food_packs

    for z in zone_weights:
        b = int(boat_alloc[z])
        m = int(medic_alloc[z])
        f = int(food_alloc[z])
        response[z]["boats"] = b
        response[z]["medics"] = m
        response[z]["food_packs"] = f
        remaining_boats -= b
        remaining_medics -= m
        remaining_food -= f

    # Step 4: Distribute leftover resources to top-weighted zones
    sorted_zones = sorted(zone_weights.items(), key=lambda x: x[1], reverse=True)

    for z, _ in sorted_zones:
        if remaining_boats > 0:
            response[z]["boats"] += 1
            remaining_boats -= 1
        if remaining_medics > 0:
            response[z]["medics"] += 1
            remaining_medics -= 1
        if remaining_food > 0:
            response[z]["food_packs"] += 1
            remaining_food -= 1
        if remaining_boats == 0 and remaining_medics == 0 and remaining_food == 0:
            break

    return dict(response)





In [ ]:
rule_based = rule_based_response_dict(zone_state)
print(rule_based)


{'B3': {'boats': 1, 'medics': 1, 'food_packs': 1}, 'A1': {'boats': 1, 'medics': 1, 'food_packs': 1}, 'A3': {'boats': 1, 'medics': 1, 'food_packs': 1}, 'C1': {'boats': 1, 'medics': 1, 'food_packs': 1}}


**Using open source LLM**

In [ ]:
!pip install -q transformers accelerate bitsandbytes


In [ ]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",
    device_map="auto",
    torch_dtype="auto"
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Build the prompt
prompt = (
    "You are a flood disaster response AI.\n"
    "Your job is to allocate the following resources:\n"
    "- 4 boats\n- 4 medics\n- 4 food packs\n"
    "Use ONLY this format in your response (JSON):\n"
    "{\n"
    '  "A1": {"boats": 1, "medics": 0, "food_packs": 1},\n'
    '  "B3": {"boats": 2, "medics": 1, "food_packs": 0},\n'
    "  ...\n"
    "}\n\n"
    "Here is the flood situation:\n"
    f"{flood_report}\n"
    "Respond only with JSON. Do not add explanations.\n"
)


In [ ]:
llm_response = generator(prompt, max_new_tokens=300, temperature=0.7)[0]['generated_text']
print("🧠 Falcon LLM Response:\n")
print(llm_response)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


🧠 Falcon LLM Response:

You are a flood disaster response AI.
Your job is to allocate the following resources:
- 4 boats
- 4 medics
- 4 food packs
Use ONLY this format in your response (JSON):
{
  "A1": {"boats": 1, "medics": 0, "food_packs": 1},
  "B3": {"boats": 2, "medics": 1, "food_packs": 0},
  ...
}

Here is the flood situation:
Current Flood Situation Report:
- Zone A1: Status - FLOODED, Water Level - 3, People Trapped - 13, Boats - 0, Medics - 0, Food Packs - 0
- Zone A2: Status - SAFE, Water Level - 1, People Trapped - 7, Boats - 0, Medics - 0, Food Packs - 0
- Zone A3: Status - AT-RISK, Water Level - 2, People Trapped - 51, Boats - 0, Medics - 0, Food Packs - 0
- Zone B1: Status - AT-RISK, Water Level - 2, People Trapped - 22, Boats - 0, Medics - 0, Food Packs - 0
- Zone B2: Status - SAFE, Water Level - 1, People Trapped - 24, Boats - 0, Medics - 0, Food Packs - 0
- Zone B3: Status - FLOODED, Water Level - 3, People Trapped - 45, Boats - 0, Medics - 0, Food Packs - 0
- Zone C

1) Improve the data set and inlcude more paramters
2)fine tune the model
3)use neural network